# pytorch-yolo2

### create the subfolder

In [ ]:
! mkdir onnx

ref: https://github.com/longcw/yolo2-pytorch

### download weight

In [ ]:
! wget http://pjreddie.com/media/files/yolo.weights

### try predict

In [1]:
! python detect.py './cfg/yolo.cfg' './yolo.weights' './data/dog.jpg'

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
   11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 256
   12 conv    512  3 x 3 / 1    26 x  26 x 256   ->    26 x  26 x 512
   13 conv    256  1 x 

### get model

In [2]:
from darknet import Darknet

In [3]:
cfgfile =  './cfg/yolo.cfg' 
weightfile =  './yolo.weights'
imgfile = './data/dog.jpg'

In [4]:
m = Darknet(cfgfile)
m.load_weights(weightfile)
print('Loading weights from %s... Done!' % (weightfile))

Loading weights from ./yolo.weights... Done!


cfg.py:175: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w
cfg.py:157: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w


### save detection information

In [5]:
import pickle
op_dict = {
    'num_classes':m.num_classes,
    'anchors':m.anchors,
    'num_anchors':m.num_anchors
}
pickle.dump(op_dict, open('detection_information.pkl','wb'))

# use Onnx to convert model

ref: https://github.com/onnx/tutorials/blob/master/tutorials/PytorchOnnxExport.ipynb

In [6]:
import torch.onnx
from torch.autograd import Variable

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = Variable(torch.randn(1, 3, 416, 416))
# Obtain your model, it can be also constructed in your script explicitly
model = m
# Invoke export
torch.onnx.export(model, dummy_input, "onnx/yolo2.onnx")